# IMPORTS 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

In [ ]:
pip install ultralytics


# CONFIG CLASS 

In [ ]:
from pathlib import Path
from ultralytics import YOLO
import torch

# === Define Paths ===
ROOT = Path.cwd()
CKPT = ROOT / "artifacts" / "model" / "openthermalpose_yolov8_pose" / "weights" / "last.pt"
DATA = ROOT / "artifacts" / "OPEN_THERMAL_IMAGE" / "openthermalpose.yaml"
TRAIN_PATH = Path("/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/train/images")
VAL_PATH = Path("/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/images")


In [ ]:
from pathlib import Path

class Config:
    def __init__(self):
        #self.dataset_path = Path("../artifacts/OPEN_THERMAL_IMAGE").resolve()
        self.yaml_path = DATA
        self.model_save_dir = Path("../artifacts/model/openthermalpose").resolve()
        self.model_save_dir.mkdir(parents=True, exist_ok=True)

        self.epochs = 10
        self.img_size = 640
        self.batch_size = 8
        self.learning_rate = 0.001
        self.num_keypoints = 17
        self.device = "cpu"  # or "cuda" if GPU available


In [ ]:
import yaml

In [ ]:
def create_yaml(cfg: Config):
    train_images = TRAIN_PATH
    val_images = VAL_PATH

    data_yaml = {
        'train': str(train_images),
        'val': str(val_images),
        'nc': 1,
        'names': ['person'],
        'keypoints': [f'kp{i}' for i in range(cfg.num_keypoints)],
        'kpt_shape': [cfg.num_keypoints, 3]  # important for pose!
    }

    with open(cfg.yaml_path, "w") as f:
        yaml.dump(data_yaml, f)
    print(f"Dataset YAML with kpt_shape created at {cfg.yaml_path}")


# TRAINING CLASS 

In [ ]:
from pathlib import Path

dataset_folder = Path("./artifacts/OPEN_THERMAL_IMAGE").resolve()
print("Resolved dataset folder:", dataset_folder)

train_images = dataset_folder / "train" / "images"
val_images = dataset_folder / "val" / "images"

print("Train Path:", train_images, "Exists:", train_images.exists())
print("Val Path:", val_images, "Exists:", val_images.exists())


In [ ]:
from pathlib import Path
import yaml

def create_openthermalpose_yaml(dataset_dir: str, num_keypoints=17):
    dataset_path = Path(dataset_dir).resolve()
    train_images = dataset_path / "train" / "images"
    val_images = dataset_path / "val" / "images"

    assert train_images.exists(), f"Train images folder not found: {train_images}"
    assert val_images.exists(), f"Val images folder not found: {val_images}"

    data_yaml = {
        'train': str(train_images),
        'val': str(val_images),
        'nc': 1,  # number of classes (person)
        'names': ['person'],
        'keypoints': [f'kp{i}' for i in range(num_keypoints)]
    }

    yaml_file = dataset_path / "openthermalpose.yaml"
    with open(yaml_file, "w") as f:
        yaml.dump(data_yaml, f)

    print(f"YOLOv8 dataset yaml file created at: {yaml_file}")
    return yaml_file

# Example usage
#dataset_folder = "../artifacts/OPEN_THERMAL_IMAGE"  # relative path from your notebook
yaml_path = create_openthermalpose_yaml(dataset_folder)


In [ ]:
from ultralytics import YOLO

class YOLOv8PoseTrainer:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.model = YOLO("yolov8n-pose.pt")  # pre-trained pose model checkpoint

    def train(self):
        results = self.model.train(
            data=str(self.cfg.yaml_path),
            epochs=self.cfg.epochs,
            imgsz=self.cfg.img_size,
            batch=self.cfg.batch_size,
            lr0=self.cfg.learning_rate,
            device=self.cfg.device,
            name="openthermalpose_yolov8_pose",
            project=str(self.cfg.model_save_dir.parent),  # save in 'artifacts/model/'
            exist_ok=True,
            verbose=True
        )
        print("Training complete.")
        # Save best weights manually if you want
        weights_path = self.cfg.model_save_dir / "best.pt"
        self.model.model.save(weights_path)
        print(f"Model saved to {weights_path}")
        return results


In [ ]:
class YOLOv8PoseEvaluator:
    def __init__(self, cfg: Config, model_path: Path):
        self.cfg = cfg
        self.model = YOLO(str(model_path))

    def evaluate(self):
        metrics = self.model.val(
            data=str(self.cfg.yaml_path),
            imgsz=self.cfg.img_size,
            device=self.cfg.device,
            batch=self.cfg.batch_size,
            verbose=True
        )
        print("Evaluation complete.")
        print(metrics)
        return metrics


# MAIN PIPELINE 

In [ ]:
def main():
    cfg = Config()
    create_yaml(cfg)

    trainer = YOLOv8PoseTrainer(cfg)
    trainer.train()

    model_path = cfg.model_save_dir / "best.pt"
    evaluator = YOLOv8PoseEvaluator(cfg, model_path)
    evaluator.evaluate()

if __name__ == "__main__":
    main()


In [ ]:
from pathlib import Path
print(Path("artifacts/OPEN_THERMAL_IMAGE/openthermalpose.yaml").resolve().exists())


# code for resuming training 

In [ ]:
from pathlib import Path
from ultralytics import YOLO

ROOT = Path.cwd()
CKPT = ROOT / "artifacts" / "model" / "openthermalpose_yolov8_pose" / "weights" / "last.pt"
DATA = ROOT / "artifacts" / "OPEN_THERMAL_IMAGE" / "openthermalpose2.yaml"

print("📁 ROOT dir:", ROOT)
print("🔍 Looking for checkpoint at:", CKPT)
print("✅ Exists:", CKPT.exists())

if CKPT.exists():
    print(f"✅ Resuming from checkpoint: {CKPT}")
    model = YOLO(str(CKPT))
    resume_flag = True
else:
    raise FileNotFoundError(f"❌ Checkpoint not found at: {CKPT}")

# === Dataset Paths from YAML ===
train_path = Path("/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/train/images")
val_path = Path("/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/images")

# === Check if dataset paths exist ===
print(f"📁 Checking if training images path exists: {train_path} -> {train_path.exists()}")
print(f"📁 Checking if validation images path exists: {val_path} -> {val_path.exists()}")

if not train_path.exists():
    raise FileNotFoundError(f"❌ Training path not found: {train_path}")

if not val_path.exists():
    raise FileNotFoundError(f"❌ Validation path not found: {val_path}")


📁 ROOT dir: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset
🔍 Looking for checkpoint at: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/model/openthermalpose_yolov8_pose/weights/last.pt
✅ Exists: True
✅ Resuming from checkpoint: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/model/openthermalpose_yolov8_pose/weights/last.pt


In [19]:
from pathlib import Path
from ultralytics import YOLO
import torch

# === Define Paths ===
ROOT = Path.cwd()
CKPT = ROOT / "artifacts" / "model" / "openthermalpose_yolov8_pose" / "weights" / "last.pt"
DATA = ROOT / "artifacts" / "OPEN_THERMAL_IMAGE" / "openthermalpose2.yaml"
TRAIN_PATH = Path("/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/train/images")
VAL_PATH = Path("/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/images")

# === Detect best available device ===
if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

# === Checkpoint & Dataset Path Validations ===
print(f"📂 ROOT dir: {ROOT}")
print(f"🔍 Looking for checkpoint at: {CKPT}")
print(f"✅ Checkpoint exists: {CKPT.exists()}")
print(f"📁 Train path exists: {TRAIN_PATH.exists()}")
print(f"📁 Val path exists: {VAL_PATH.exists()}")

if not TRAIN_PATH.exists() or not VAL_PATH.exists():
    raise FileNotFoundError("❌ One or both dataset paths do not exist. Please check train/val image folders.")

# === Load model ===
try:
    if CKPT.exists():
        print(f"✅ Resuming from checkpoint: {CKPT}")
        model = YOLO(str(CKPT))
        resume_flag = True
    else:
        raise FileNotFoundError(f"❌ Checkpoint not found: {CKPT}")
except Exception as e:
    print(f"🚨 Error loading checkpoint: {e}")
    print("⚠️ Falling back to base YOLOv8n-pose model...")
    model = YOLO("yolov8n-pose.pt")
    resume_flag = False

# === Start Training ===
try:
    model.train(
        data=str(DATA),
        epochs=10,
        resume=resume_flag,
        device=DEVICE,
        project=str(ROOT / "artifacts/model"),
        name="openthermalpose_yolov8_pose"
    )
except RuntimeError as e:
    if "knet" in str(e).lower():
        print("❌ knet error: This may be due to unsupported GPU backend.")
        print("👉 Try setting `device='cpu'` or rechecking your PyTorch/MPS environment.")
    raise e


📂 ROOT dir: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset
🔍 Looking for checkpoint at: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/model/openthermalpose_yolov8_pose/weights/last.pt
✅ Checkpoint exists: True
📁 Train path exists: True
📁 Val path exists: True
✅ Resuming from checkpoint: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/model/openthermalpose_yolov8_pose/weights/last.pt
Ultralytics 8.3.152 🚀 Python-3.12.7 torch-2.7.0 MPS (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/openthermalpose.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.0, flipud=0.0, format=torchscript, 

train: Scanning /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/train/labels.cache... 8846 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8846/8846 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 373.1±139.6 MB/s, size: 157.1 KB)



val: Scanning /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/labels.cache... 934 images, 0 backgrounds, 934 corrupt: 100%|██████████| 934/934 [00:00<?, ?it/s]

val: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_1_450_1.png: ignoring corrupt image/label: labels require 39 columns each
val: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_2_450_1.png: ignoring corrupt image/label: labels require 39 columns each
val: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_3_450_1.png: ignoring corrupt image/label: labels require 39 columns each
val: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_4_450_1.png: ignoring corrupt image/label: labels require 39 columns each
val: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_5_450_1.png: ignoring corrupt image/label: labels require 39 columns each
val: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_6_1_1.png: ignoring corrupt image/label: labels 

RuntimeError: No valid images found in /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/labels.cache. Images with incorrectly formatted labels are ignored. See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [ ]:
import os
from pathlib import Path

# Define path to the val label directory
label_root = Path("/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/OPEN_THERMAL_IMAGE/val/labels")

for label_path in label_root.glob("*.txt"):
    with open(label_path) as f:
        lines = f.readlines()

    new_lines = []
    modified = False

    for line in lines:
        parts = line.strip().split()
        try:
            floats = list(map(float, parts))
            class_id = int(floats[0])
            coords = floats[1:]

            # Clamp each coordinate to [0.0, 1.0]
            normalized_coords = [min(max(c, 0.0), 1.0) for c in coords]
            if coords != normalized_coords:
                modified = True

            new_line = f"{class_id} " + " ".join(f"{val:.6f}" for val in normalized_coords)
            new_lines.append(new_line + "\n")
        except:
            print(f"⚠️ Skipping invalid line in {label_path.name}: {line.strip()}")

    if modified:
        with open(label_path, "w") as f:
            f.writelines(new_lines)
        print(f"🔧 Normalized: {label_path.name}")
